<font size = 32> Juvo test </font>

# Import libraries

In [7]:
#Data manipulation libs
import pandas as pd
import numpy as np

#Machine Learning libs
import sklearn
import xgboost as xbg

#Aux libs
from functools import reduce

%reload_ext watermark
%watermark --iversions

pandas : 1.3.2
sklearn: 0.24.1
numpy  : 1.19.5
xgboost: 1.6.1



# Functions

In [16]:
def variable_summary(series: pd.Series):
    """
    Series describer
    
    Parameters
    ----------
    :series: pandas Series to analyse 
    """
    variable = series.name
    dtype = series.dtype
    na = series.isna().sum()
    na_pct = str(round(100*na/series.shape[0],2))+"%"
    if('object' in str(dtype)):
        first_element = series.pipe(lambda x: x[x==x])
        if(first_element.shape[0] == 0):
            unique = series.nunique()
        else:
            element_type = str(type(first_element.iloc[0]))
            if((element_type == "<class 'numpy.ndarray'>") | (element_type == "<class 'list'>") | (element_type == "<class 'set'>") | (element_type == "<class 'dict'>")):
                unique = '-'
            else:
                unique = series.nunique()
    else:
        unique = series.nunique()
    
    
    if( ('object' in str(dtype)) | ('category' in str(dtype)) ):
        minimum,mean,maximum,std,quat25,quat75,median,skew,kurt,mean_std = ("-","-","-","-","-","-","-","-","-","-")
    else:
        minimum = series.min()
        mean = series.mean()
        maximum = series.max()
        std = series.std()
        if('bool' in str(dtype)):
            quat25 = series.astype(float).quantile(0.25)
            quat75 = series.astype(float).quantile(0.75)
            median = series.astype(float).quantile(0.5)
        else:
            quat25 = series.quantile(0.25)
            quat75 = series.quantile(0.75)
            median = series.quantile(0.5)
        
        if( ('int' in str(dtype)) | ('float' in str(dtype)) ):
            skew = round(series.skew(),2) #skewness
            kurt = round(series.kurt(),2) #kurtosis
            if((all(series == 0)) or (std == 0)):
                mean_std = 0
            else:
                mean_std = round(mean/std,2)
        else:
            skew,kurt,mean_std = ("-","-","-")
        
        
    return(pd.DataFrame({'variable':[variable],
                        'type':[dtype],
                        'na':[na],
                        'na_pct':[na_pct],
                        'unique':[unique],
                        'min':[minimum],
                        'quat25':[quat25],
                        'median':[median],
                        'mean':[mean],
                        'quat75':[quat75],
                        'max':[maximum],
                        'std':[std],
                        'skewness':[skew],
                        'kurtosis':[kurt],
                        'media_desvio':[mean_std]}))

#Data describer function
def describe(df):
    """
    Dataframe describer
    
    Parameters
    ----------
    :df: pandas DataFrame to analyse 
    """
    print("Number of Rows:",df.shape[0])
    describe_dataset = reduce(lambda x,y: pd.concat((x,y)), [variable_summary(df[col]) for col in df.columns])
    return describe_dataset.reset_index(drop=True)


# Import data

In [31]:
loans = pd.read_csv("../data/Brazil_DS_loans_2019-11-10_2019-12-05.csv")
prev  = pd.read_csv("../data/Brazil_DS_prev_loans.csv")
recharge = pd.read_csv("../data/Brazil_DS_recharges_2019-08-10_2019-12-05.csv")

In [32]:
#Convert into datetime
prev.created_at = pd.to_datetime(prev.created_at)
loans.created_at = pd.to_datetime(loans.created_at)
recharge.recharge_timestamp = pd.to_datetime(recharge.recharge_timestamp)

In [33]:
#Loans description
#There are only 2 different loan amount
#On this one, only 18.89%% did not paided
describe(loans)

Number of Rows: 12926


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,uuid,object,0,0.0%,9513,-,-,-,-,-,-,-,-,-,-
1,loan_id,int64,0,0.0%,12926,16513,19745.25,22986.5,22984.485842,26218.75,29454,3736.246948,-0.0,-1.2,6.15
2,amount,float64,0,0.0%,2,5.0,5.0,5.0,5.449095,5.0,10.0,1.429666,2.87,6.24,3.81
3,created_at,datetime64[ns],0,0.0%,12926,2019-11-10 00:04:42.327443,2019-11-18 00:32:10.321530624,2019-11-24 13:47:35.415590400,2019-11-23 19:56:40.806889472,2019-11-29 15:08:45.370689024,2019-12-04 23:54:57.280521,6 days 22:38:10.394918360,-,-,-
4,paid_at,object,2442,18.89%,10484,-,-,-,-,-,-,-,-,-,-


In [34]:
#Prev description
#There are only 4 different loan amount
#On this one, only 0.58% did not paided
#less users than loans dataset
describe(prev)

Number of Rows: 6035


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,uuid,object,0,0.0%,1964,-,-,-,-,-,-,-,-,-,-
1,loan_id,int64,0,0.0%,6035,1,6669.0,10455.0,9932.314499,13641.5,16504,4315.519482,-0.37,-0.89,2.3
2,amount,float64,0,0.0%,4,3.85,5.0,5.0,5.77029,5.0,10.0,1.805531,1.92,1.67,3.2
3,created_at,datetime64[ns],0,0.0%,6035,2019-01-16 23:41:24.785649,2019-09-20 13:26:54.973658112,2019-10-12 09:15:09.591973888,2019-10-04 06:41:34.816445696,2019-10-28 16:23:30.478653440,2019-11-09 23:07:27.962833,32 days 11:29:48.450471939,-,-,-
4,paid_at,object,35,0.58%,6000,-,-,-,-,-,-,-,-,-,-


In [35]:
#recharge description
describe(recharge)

Number of Rows: 94793


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,uuid,object,0,0.0%,9431,-,-,-,-,-,-,-,-,-,-
1,recharge_timestamp,datetime64[ns],0,0.0%,94058,2019-08-10 00:04:21,2019-09-19 21:42:41,2019-10-23 11:31:45,2019-10-17 19:35:46.300845312,2019-11-16 15:59:02,2019-12-04 23:53:01,33 days 22:34:54.137257993,-,-,-
2,recharge_value,float64,0,0.0%,15,3.0,5.0,10.0,11.328081,15.0,100.0,6.757032,1.7,6.07,1.68
3,balance_after_recharge,float64,52,0.05%,3175,0.0,6.07,10.27,12.432254,15.65,264.44,7.862897,4.54,83.96,1.58
4,institution_name,object,35412,37.36%,114,-,-,-,-,-,-,-,-,-,-
5,institution_type,object,35804,37.77%,9,-,-,-,-,-,-,-,-,-,-


# Data analysis

According to the explanation: "_A loan is considered repaid if it's paid within 60 days_".<br> 
Besides, "_a user pays for loans by making recharges after taking a loan_"<br>
Let's see this behaviour taking a user

In [108]:
loans.query("paid_at != paid_at").head(10)

,uuid,loan_id,amount,created_at,paid_at
1,b8d0387ac545467fa37c51ac6c104fed,26407,5.0,2019-11-29 19:17:10.097070,NaN
10,9f3c1a0325d24a339c454e6b847f012e,29334,5.0,2019-12-04 20:04:01.619980,NaN
20,8c4d79a684b043149dcd2aef35d5b266,26515,5.0,2019-11-29 21:55:03.081059,NaN
29,71ca719598d24dc7822626a56fe328f9,20075,5.0,2019-11-18 17:31:35.528430,NaN
30,5dcfd193d7654a02baf3606e42a3bc1f,16682,5.0,2019-11-10 21:30:48.835132,NaN
33,6ebd721b3edc49a386ee778e0c5dfa45,29153,5.0,2019-12-04 14:39:10.932118,NaN
34,7faa3668919145fdbb703ab80bdd60ff,17169,5.0,2019-11-12 15:38:30.448831,NaN
37,6a395b43f5c44802a0b205cd7ccde55e,29139,5.0,2019-12-04 14:17:06.868078,NaN
40,975008f5602f442ea9f2fc2885286d37,17370,5.0,2019-11-12 21:10:51.004919,NaN
48,4708792e9c744a1abc3f5775ab2b2b33,18328,5.0,2019-11-14 17:57:38.927132,NaN


In [25]:
prev.query("paid_at != paid_at").head(10)

,uuid,loan_id,amount,created_at,paid_at
116,fa19674e1eb344f8b19591c493a4557b,1371,5.0,2019-07-12 09:37:46.031214,NaN
483,a66c6e2d6b6540c6b2f6a136def16cd1,1269,5.0,2019-07-10 02:15:15.857633,NaN
574,1e449a1ac95a49c3b8353dd05749d78d,2426,5.0,2019-08-02 23:39:21.905010,NaN
621,1dd011320a994ef7a4453f5745d7ce4f,1487,5.0,2019-07-14 22:01:48.230560,NaN
912,67b98f4c0c66470198a2f7bd179b3b97,648,5.0,2019-06-25 13:55:48.236896,NaN
1155,4f73292fb7864e41a9853967789e64b3,936,5.0,2019-07-02 12:42:29.478425,NaN
1182,d72c0910dba4436fa420df0a59adeb1a,2688,5.0,2019-08-07 21:22:52.958146,NaN
1559,56c089f373fc4f7bb46c5e56f11edaa1,382,5.0,2019-06-15 01:18:59.349732,NaN
1605,95f77a808cf240339a4c7bfbe903970d,622,5.0,2019-06-24 18:58:18.449026,NaN
1809,9f58366573394e4587675d5cd7188adc,726,5.0,2019-06-27 12:41:36.313722,NaN


In this following example, even after the user recharging the loan 1371, the time between the loan and the first recharge is over 60 days, so the loan is considered not payed. However, considering the loan 23811, the payment occured a few days after that

In [54]:
uuid = 'fa19674e1eb344f8b19591c493a4557b'
pd.concat((prev.query("'%s' == uuid"%uuid).assign(DF='prev'),
           loans.query("'%s' == uuid"%uuid).assign(DF = 'loans'))).sort_values('created_at')

,uuid,loan_id,amount,created_at,paid_at,DF
116,fa19674e1eb344f8b19591c493a4557b,1371,5.0,2019-07-12 09:37:46.031214,NaN,prev
5221,fa19674e1eb344f8b19591c493a4557b,23811,5.0,2019-11-26 04:09:43.098861,2019-11-30 01:04:05.827473,loans


In [55]:
recharge.query("'%s' == uuid"%uuid).sort_values("recharge_timestamp")

,uuid,recharge_timestamp,recharge_value,balance_after_recharge,institution_name,institution_type
38027,fa19674e1eb344f8b19591c493a4557b,2019-10-14 23:23:29,10.0,10.09,M4U URA-USSD,Distribuidor Nacional
80222,fa19674e1eb344f8b19591c493a4557b,2019-11-26 01:09:43,5.0,5.10,NaN,NaN
83980,fa19674e1eb344f8b19591c493a4557b,2019-11-29 22:03:29,13.0,14.12,M4U URA-USSD,Distribuidor Nacional


Are the institution name and type always the same, taking the same uuid?

In [96]:
#No
def nunique(x):
    return x.nunique()
recharge[['institution_name','institution_type','uuid']].drop_duplicates().groupby("uuid").agg({"institution_name":nunique,"institution_type":nunique})

,institution_name,institution_type
uuid,,
000c20ae97084d4199e1c61a6a15a79e,0,0
000c4c43634a4e4cabee82c946481adc,4,2
00163f5a2e75448daf00186d60ce5ce7,2,2
00186a872c254b5fbd8be83f5343efec,1,1
001d161dd9854887b7d38e5f40db36c9,4,2
...,...,...
ffd5de5cc31e4627bad4db3bb86729e0,3,3
ffd62186c39c45d785b5fe15e0a79b11,5,3
ffd8a8416e824e4289e4f5d4cc69f8de,5,2


Let's create a single dataset, with the data history

In [53]:
loan_ = pd.concat((prev,loans))

In [99]:
df = pd.concat((loan_,recharge.rename(columns = {"recharge_timestamp":"created_at","recharge_value":"amount"})))\
    .sort_values(["uuid","created_at"]).reset_index(drop=True)

In [111]:
#If loan_id is NULL, than it is a recharge. Otherwise, it is a loan
df.query("'7faa3668919145fdbb703ab80bdd60ff' == uuid")

,uuid,loan_id,amount,created_at,paid_at,balance_after_recharge,institution_name,institution_type
55494,7faa3668919145fdbb703ab80bdd60ff,NaN,5.0,2019-11-12 12:38:31.000000,NaN,5.01,NaN,NaN
55495,7faa3668919145fdbb703ab80bdd60ff,17169.0,5.0,2019-11-12 15:38:30.448831,NaN,NaN,NaN,NaN


Some feature we should create:
* loan amount until now
* recharge amount until now
* recharge mean until now
* loan mean until now
* not payed loan amount until now
* payed loan times after the not payed
* days since the last not payed loan
* days since the last recharge
* days since the last loan
* days between now and the first recharge
* last instituition type
* most commum instituition type
* balance_after_recharge sum until now

Disconsider the last 60 days of the dataset